# **DATA PROCESSING**

In [1]:
#%run '/home/cyberpunk/Trading_bot_v1.0/notebooks/Data_collection_and_Preprocessing.ipynb'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
from datetime import datetime
from scipy.stats import zscore
import talib

In [3]:
# loading the data

In [4]:
# Import the necessary library
import pandas as pd

# Set the file path
file_path = '/home/cyberpunk/Trading_bot_v1.0/Data/Preprocessed/XAU_1h_preprocessed_2004_to_2024.csv'

# Load the data into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to check
data.head()

,Unnamed: 0,Open,High,Low,Close,Volume,Time_Diff,Volume_Zscore,50_rolling_avg
0,2004-06-11 07:00:00,384.0,384.3,383.3,383.8,44.0,NaN,-0.730712,NaN
1,2004-06-11 08:00:00,383.8,384.3,383.1,383.1,41.0,1.0,-0.731559,NaN
2,2004-06-11 09:00:00,383.1,384.1,382.8,383.1,55.0,1.0,-0.727607,NaN
3,2004-06-11 10:00:00,383.0,383.8,383.0,383.6,33.0,1.0,-0.733817,NaN
4,2004-06-11 11:00:00,383.6,383.8,383.5,383.6,23.0,1.0,-0.736640,NaN


In [5]:
# Convert the "Unnamed: 0" column to datetime
data['Time'] = pd.to_datetime(data['Unnamed: 0'])

# Drop the "Unnamed: 0" column as it's now redundant
data = data.drop(columns=['Unnamed: 0'])

# Set 'Time' as the index for time-series analysis
data.set_index('Time', inplace=True)

# Check the updated DataFrame
data.head()

,Open,High,Low,Close,Volume,Time_Diff,Volume_Zscore,50_rolling_avg
Time,,,,,,,,
2004-06-11 07:00:00,384.0,384.3,383.3,383.8,44.0,NaN,-0.730712,NaN
2004-06-11 08:00:00,383.8,384.3,383.1,383.1,41.0,1.0,-0.731559,NaN
2004-06-11 09:00:00,383.1,384.1,382.8,383.1,55.0,1.0,-0.727607,NaN
2004-06-11 10:00:00,383.0,383.8,383.0,383.6,33.0,1.0,-0.733817,NaN
2004-06-11 11:00:00,383.6,383.8,383.5,383.6,23.0,1.0,-0.736640,NaN


In [6]:
# Check for missing values
#data.isnull().sum()
#data['Time_Diff'] = data['Time_Diff'].fillna(method='ffill')

## **FEATURE ENGINEERING**

### Lag Features (1-hour, 2-hour, 3-hour Lag)

In [7]:
# Create lagged features for the Close price
data['Close_t-1'] = data['Close'].shift(1)
data['Close_t-2'] = data['Close'].shift(2)
data['Close_t-3'] = data['Close'].shift(3)

# Check for NaN values introduced by the lag
data.head()

,Open,High,Low,Close,Volume,Time_Diff,Volume_Zscore,50_rolling_avg,Close_t-1,Close_t-2,Close_t-3
Time,,,,,,,,,,,
2004-06-11 07:00:00,384.0,384.3,383.3,383.8,44.0,NaN,-0.730712,NaN,NaN,NaN,NaN
2004-06-11 08:00:00,383.8,384.3,383.1,383.1,41.0,1.0,-0.731559,NaN,383.8,NaN,NaN
2004-06-11 09:00:00,383.1,384.1,382.8,383.1,55.0,1.0,-0.727607,NaN,383.1,383.8,NaN
2004-06-11 10:00:00,383.0,383.8,383.0,383.6,33.0,1.0,-0.733817,NaN,383.1,383.1,383.8
2004-06-11 11:00:00,383.6,383.8,383.5,383.6,23.0,1.0,-0.736640,NaN,383.6,383.1,383.1


### Calculate Returns (Percentage Change in 1 Hour)

In [8]:
# Calculate hourly returns (percentage change)
data['Returns_t'] = data['Close'].pct_change()

# Check the results
data[['Close', 'Returns_t']].head()

,Close,Returns_t
Time,,
2004-06-11 07:00:00,383.8,NaN
2004-06-11 08:00:00,383.1,-0.001824
2004-06-11 09:00:00,383.1,0.000000
2004-06-11 10:00:00,383.6,0.001305
2004-06-11 11:00:00,383.6,0.000000


### Step 4: Rolling Averages (5, 10, 20 Hours)

In [9]:
# 5-hour, 10-hour, and 20-hour rolling averages
data['5_rolling_avg'] = data['Close'].rolling(window=5).mean()
data['10_rolling_avg'] = data['Close'].rolling(window=10).mean()
data['20_rolling_avg'] = data['Close'].rolling(window=20).mean()

# Check the results
data[['Close', '5_rolling_avg', '10_rolling_avg', '20_rolling_avg']].head()

,Close,5_rolling_avg,10_rolling_avg,20_rolling_avg
Time,,,,
2004-06-11 07:00:00,383.8,NaN,NaN,NaN
2004-06-11 08:00:00,383.1,NaN,NaN,NaN
2004-06-11 09:00:00,383.1,NaN,NaN,NaN
2004-06-11 10:00:00,383.6,NaN,NaN,NaN
2004-06-11 11:00:00,383.6,383.44,NaN,NaN


### Rolling Volatility (10 and 20 Hour Windows)

In [10]:
# 10-hour and 20-hour rolling standard deviation (volatility)
data['Rolling_Volatility_10'] = data['Close'].rolling(window=10).std()
data['Rolling_Volatility_20'] = data['Close'].rolling(window=20).std()

# Check the results
data[['Close', 'Rolling_Volatility_10', 'Rolling_Volatility_20']].head()

,Close,Rolling_Volatility_10,Rolling_Volatility_20
Time,,,
2004-06-11 07:00:00,383.8,NaN,NaN
2004-06-11 08:00:00,383.1,NaN,NaN
2004-06-11 09:00:00,383.1,NaN,NaN
2004-06-11 10:00:00,383.6,NaN,NaN
2004-06-11 11:00:00,383.6,NaN,NaN


### Rate of Change (ROC) for Momentum (10 Hours)

In [11]:
import talib

# Calculate Rate of Change (ROC) for momentum (e.g., 10-hour period)
data['ROC_10'] = talib.ROC(data['Close'], timeperiod=10)

# Check the results
data[['Close', 'ROC_10']].head(5)

,Close,ROC_10
Time,,
2004-06-11 07:00:00,383.8,NaN
2004-06-11 08:00:00,383.1,NaN
2004-06-11 09:00:00,383.1,NaN
2004-06-11 10:00:00,383.6,NaN
2004-06-11 11:00:00,383.6,NaN


### Price Change Percentage

In [12]:
# Price change percentage from the previous hour
data['Price_Change_Percent'] = data['Close'].pct_change() * 100

### Bollinger Bands

In [13]:
# Calculate Bollinger Bands (20-period by default)
window = 20
data['Rolling_Mean'] = data['Close'].rolling(window=window).mean()
data['Rolling_STD'] = data['Close'].rolling(window=window).std()

# Upper and Lower Bands
data['Bollinger_Upper'] = data['Rolling_Mean'] + (2 * data['Rolling_STD'])
data['Bollinger_Lower'] = data['Rolling_Mean'] - (2 * data['Rolling_STD'])

### Exponential Moving Average (EMA)

In [14]:
# Calculate EMA (12-period and 26-period)
data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()

### Relative Strength Index (RSI)

In [15]:
import talib

# Calculate RSI (14-period by default)
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)

### Moving Average Convergence Divergence (MACD)

In [16]:
# Calculate MACD and Signal Line
data['MACD'], data['MACD_Signal'], _ = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

### Stochastic Oscillator

In [17]:
# Calculate Stochastic Oscillator (14-period by default)
data['SlowK'], data['SlowD'] = talib.STOCH(data['High'], data['Low'], data['Close'], fastk_period=14, slowk_period=3, slowd_period=3)

## Handling NaN Values

In [18]:
# Drop rows with NaN values
data = data.dropna()

### Saving the processed data

In [19]:
# Saving the processed data as 'XAU_1h_processed_2004_to_2024.csv'
processed_file_path = '/home/cyberpunk/Trading_bot_v1.0/Data/Processed/XAU_1h_processed_2004_to_2024.csv'
data.to_csv(processed_file_path)

# Verify that the data is saved
print(f"Processed data saved to {processed_file_path}")

Processed data saved to /home/cyberpunk/Trading_bot_v1.0/Data/Processed/XAU_1h_processed_2004_to_2024.csv
